In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np
from string import digits
from collections import Counter
from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from keras.src.utils.np_utils import to_categorical

In [2]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:
data_train = pd.read_csv("vlsp_sentiment_train.csv", sep=',')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv("vlsp_sentiment_test.csv", sep=',')
data_test.columns =['Class', 'Data']

In [4]:
print(data_train.shape)
print(data_test.shape)
pd.set_option('max_colwidth', 100)
print(data_train.head())

(5100, 2)
(1050, 2)
   Class  \
0     -1   
1     -1   
2     -1   
3     -1   
4     -1   

                                                                                                  Data  
0  Mình đã dùng anywhere thế hệ đầu, quả là đầy thất vọng, hiện tại đang vứt xó. Giá thì đắt, ngốn ...  
1  Quan tâm nhất là độ trễ có cao không, dùng thi thoảng nó cứ trễ bực mình, đấy mới chỉ là dùng vă...  
2                     dag xài con cùi bắp 98k....pin trâu, mỗi tội đánh liên minh ức chế đập hết 2 con  
3  logitech chắc hàng phải tiền triệu trở lên dùng mới thích chứ em dùng con có 400k thấy được vài ...  
4  Đang xài con m175 cùi mía , nhà xài nhiều chuột nên thử con này con kia chơi và kết quả là sau 3...  


In [5]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values

In [6]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)  

In [7]:
reviews_processed = []
unlabeled_processed = [] 
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    reviews_processed.append(review_cool_one)

In [11]:
#Use PyVi for Vietnamese word tokenizer
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())
   

In [12]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [14]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index
print(word_index)

{'.': 1, ',': 2, 'không': 3, 'là': 4, 'có': 5, 'thì': 6, 'mình': 7, 'được': 8, 'của': 9, 'và': 10, 'mà': 11, 'dùng': 12, 'mua': 13, 'cái': 14, 'này': 15, 'cũng': 16, 'hơn': 17, 'quá': 18, 'với': 19, 'bạn': 20, 'cho': 21, 'con': 22, 'rồi': 23, 'nó': 24, '!': 25, ')': 26, 'nhưng': 27, 'thấy': 28, 'còn': 29, 'giá': 30, 'như': 31, '...': 32, 'chỉ': 33, 'em': 34, 'phải': 35, 'để': 36, 'gì': 37, 'máy': 38, 'các': 39, 'đẹp': 40, 'rất': 41, 'mới': 42, 'đã': 43, 'nên': 44, 'lại': 45, 'ra': 46, ':': 47, '?': 48, 'pin': 49, 'nhiều': 50, 'thôi': 51, 'về': 52, 'khi': 53, 'vẫn': 54, 'xài': 55, 'hay': 56, 'người': 57, 'sẽ': 58, 's': 59, 'tốt': 60, 'đó': 61, 'đang': 62, '-': 63, 'iphone': 64, 'đi': 65, 'tôi': 66, '(': 67, 'làm': 68, 'trên': 69, 'chưa': 70, 'điện_thoại': 71, 'bị': 72, 'nếu': 73, 'mấy': 74, 'bác': 75, 'màn_hình': 76, 'nữa': 77, 'nào': 78, 'trong': 79, 'hàng': 80, 'lên': 81, 'ở': 82, 'vì': 83, 'samsung': 84, 'ngon': 85, 'nhìn': 86, 'đến': 87, 'biết': 88, 'nhất': 89, 'từ': 90, 'vào': 91, 

In [15]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [16]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (5100, 300)
Shape of label train and validation tensor: (5100, 3)


In [17]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('vi-model-CBOW.bin', binary=True)


vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
print("vocab size", vocabulary_size)
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

vocab size 7908


In [19]:
from keras.models import Model
from keras.layers import Input, Reshape, Conv1D, MaxPool1D, concatenate, Flatten, Dropout, Dense, LSTM
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras import regularizers
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)

################## LSTM ONLY ###############################
# reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

################# SINGLE LSTM ####################
# lstm_0 = LSTM(512)(reshape)

# YOU WANNA ADD MORE LSTM LAYERS? UNCOMMENT THIS #
# lstm_2 = LSTM(1024, return_sequences=True)(reshape)
# lstm_1 = LSTM(512, return_sequences=True)(lstm_2)
# lstm_0 = LSTM(256)(lstm_1)

############################################################


################## CRNN ####################################
reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

conv_0 = Conv1D(num_filters, (filter_sizes[0], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv1D(num_filters, (filter_sizes[1], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv1D(num_filters, (filter_sizes[2], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

conv_0 = MaxPool1D(300)(conv_0)
conv_1 = MaxPool1D(300)(conv_1)
conv_2 = MaxPool1D(300)(conv_2)
# Reshape output to match RNN dimension
conv_0 = Reshape((-1, num_filters))(conv_0)
conv_1 = Reshape((-1, num_filters))(conv_1)
conv_2 = Reshape((-1, num_filters))(conv_2)

concat = concatenate([conv_0, conv_1, conv_2])
# concat = Flatten()(concat)
lstm_0 = LSTM(512)(concat)

# YOU WANNA ADD MORE LSTM LAYERS? UNCOMMENT THIS #
# lstm_2 = LSTM(1024, return_sequences=True)(concat)
# lstm_1 = LSTM(512, return_sequences=True)(lstm_2)
# lstm_0 = LSTM(256)(lstm_1)

############################################################

dropout = Dropout(drop)(lstm_0)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)


adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 300)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 300, 400)             3163200   ['input_2[0][0]']             
                                                                                                  
 reshape_4 (Reshape)         (None, 300, 400)             0         ['embedding[1][0]']           
                                                                                                  
 conv1d_3 (Conv1D)           (None, 300, 100)             120100    ['reshape_4[0][0]']           
                                                                                              

In [ ]:
### IF YOU HAVE MODEL WEIGHT AND WANNA LOAD IT
model.load_weights("model.h5")

In [18]:
tf.debugging.set_log_device_placement(True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices()))

Num GPUs Available:  1


In [19]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

model.fit(data, labels, validation_split=0.2,
          epochs=10, batch_size=256, callbacks=callbacks_list, shuffle=True)

Epoch 1/10
16/16 [==============================] - 45s 3s/step - loss: 7.5719 - accuracy: 0.4549 - val_loss: 7.9230 - val_accuracy: 0.0324
Epoch 2/10
16/16 [==============================] - 39s 2s/step - loss: 5.4301 - accuracy: 0.6328 - val_loss: 6.1777 - val_accuracy: 0.2020
Epoch 3/10
16/16 [==============================] - 40s 3s/step - loss: 4.5598 - accuracy: 0.7159 - val_loss: 6.2032 - val_accuracy: 0.1029
Epoch 4/10
16/16 [==============================] - 40s 2s/step - loss: 3.9124 - accuracy: 0.7909 - val_loss: 5.9607 - val_accuracy: 0.0647
Epoch 5/10
16/16 [==============================] - 40s 2s/step - loss: 3.3986 - accuracy: 0.8333 - val_loss: 5.1653 - val_accuracy: 0.1137
Epoch 6/10
16/16 [==============================] - 39s 2s/step - loss: 2.9906 - accuracy: 0.8708 - val_loss: 4.8847 - val_accuracy: 0.1010
Epoch 7/10
16/16 [==============================] - 39s 2s/step - loss: 2.6193 - accuracy: 0.8953 - val_loss: 5.0855 - val_accuracy: 0.0382
Epoch 8/10
16/16 [==

In [20]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [21]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)  

In [22]:
reviews_processed_test = []
unlabeled_processed_test = [] 
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    reviews_processed_test.append(review_cool_one)

In [23]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [24]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [25]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


In [26]:
score = model.evaluate(data_test, labels_test)

33/33 [==============================] - 3s 77ms/step - loss: 2.4251 - accuracy: 0.6124


In [27]:
print("%s: %.2f" % (model.metrics_names[0], score[0]))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))


loss: 2.43
accuracy: 61.24%


In [ ]:
%cd /content

/content


In [ ]:
model.save_weights("model.h5")

In [ ]:
!cp model.h5 /content/drive/MyDrive